In [1]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig
from datasets import load_dataset, Dataset

dev = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
kuro_ds = load_dataset('Kurosawama/beam_search_DPO', split = 'train')
folio = load_dataset('yale-nlp/FOLIO', split='validation')
print(dev)

def initialize_model(model_id, llama3):
    quant_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = torch.bfloat16)
    gen_config = GenerationConfig.from_pretrained(model_id)
    if llama3:
        tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-3.2-1B')
    else:
        tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')
    tokenizer.chat_template = """
            <|im_start|>system
            {SYSTEM}<|im_end|>
            <|im_start|>user
            {INPUT}<|im_ed|>
            <|im_start|>assistant
            {OUTPUT}<|im_end|>
        """
    model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = quant_config, generation_config = gen_config).to(dev)
    model.generation_config.pad_token_id = tokenizer.pad_token_id
    return model, tokenizer

#model_id = 'meta-llama/Llama-3.2-1B'
#model_id = 'meta-llama/Llama-3.2-1B-Instruct'

base_model_id = 'meta-llama/Llama-2-7b-hf'
align_model_id = 'Kurosawama/Llama-2-7b-DPO-beamsearch-align'

base_model, base_tokenizer = initialize_model(base_model_id, False)
align_model, align_tokenizer = initialize_model(align_model_id, False)

cuda


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

C:\Users\FLopezP\Anaconda3\envs\ayuda_por_favor\lib\site-packages\peft\config.py:162: UserWarning: Unexpected keyword arguments ['alpha_pattern', 'bias', 'corda_config', 'eva_config', 'exclude_modules', 'fan_in_fan_out', 'init_lora_weights', 'layer_replication', 'layers_pattern', 'layers_to_transform', 'loftq_config', 'lora_alpha', 'lora_bias', 'lora_dropout', 'megatron_config', 'megatron_core', 'modules_to_save', 'r', 'rank_pattern', 'target_modules', 'trainable_token_indices', 'use_dora', 'use_rslora'] for class PeftConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


In [2]:
folio

Dataset({
    features: ['story_id', 'premises', 'premises-FOL', 'conclusion', 'conclusion-FOL', 'label', 'example_id'],
    num_rows: 203
})

In [3]:
def answer(prompt, model, tokenizer):
    """
    Genera la respuesta del modelo siguiendo la estrategia de beam_search.
    """
    inputs = tokenizer(prompt, return_tensors = 'pt').to(dev)
    outputs = model.generate(**inputs, max_new_tokens = 200, num_beams = 3)
    ans = tokenizer.batch_decode(outputs, skip_special_tokens = True)[0]
    ans = ans[len(prompt):]
    return ans

In [4]:
prompt_ex = """
Given a problem description and a question. The task is to parse the problem and the question into first-order logic formulars.
    The grammar of the first-order logic formular is defined as follows:
    1) logical conjunction of expr1 and expr2: expr1 ∧ expr2
    2) logical disjunction of expr1 and expr2: expr1 ∨ expr2
    3) logical exclusive disjunction of expr1 and expr2: expr1 ⊕ expr2
    4) logical negation of expr1: ¬expr1
    5) expr1 implies expr2: expr1 → expr2
    6) expr1 if and only if expr2: expr1 ↔ expr2
    7) logical universal quantification: ∀x
    8) logical existential quantification: ∃x
    --------------
    Problem:
    All people who regularly drink coffee are dependent on caffeine. People either regularly drink coffee or joke about being addicted to caffeine. No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    Predicates:
    Dependent(x) ::: x is a person dependent on caffeine.
    Drinks(x) ::: x regularly drinks coffee.
    Jokes(x) ::: x jokes about being addicted to caffeine.
    Unaware(x) ::: x is unaware that caffeine is a drug.
    Student(x) ::: x is a student.
    Premises:
    ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
    ∀x (Drinks(x) ⊕ Jokes(x)) ::: People either regularly drink coffee or joke about being addicted to caffeine.
    ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
    (Student(rina) ∧ Unaware(rina)) ⊕ ¬(Student(rina) ∨ Unaware(rina)) ::: Rina is either a student and unaware that caffeine is a drug, or neither a student nor unaware that caffeine is a drug. 
    ¬(Dependent(rina) ∧ Student(rina)) → (Dependent(rina) ∧ Student(rina)) ⊕ ¬(Dependent(rina) ∨ Student(rina)) ::: If Rina is not a person dependent on caffeine and a student, then Rina is either a person dependent on caffeine and a student, or neither a person dependent on caffeine nor a student.
    --------------
    
    Problem:
    {}
    Predicates:
"""

In [5]:
print(folio['premises'][1], '\n', folio['premises-FOL'][1])

People in this club who perform in school talent shows often attend and are very engaged with school events.
People in this club either perform in school talent shows often or are inactive and disinterested community members.
People in this club who chaperone high school dances are not students who attend the school.
All people in this club who are inactive and disinterested members of their community chaperone high school dances.
All young children and teenagers in this club who wish to further their academic careers and educational opportunities are students who attend the school. 
Bonnie is in this club and she either both attends and is very engaged with school events and is a student who attends the school or is not someone who both attends and is very engaged with school events and is not a student who attends the school. 
 ∀x (InThisClub(x) ∧ PerformOftenIn(x, schoolTalentShow) → Attend(x, schoolEvent) ∧ VeryEngagedWith(x, schoolEvent))
∀x (InThisClub(x) → PerformOftenIn(x, scho

In [ ]:
%%time
print(answer(prompt_ex.format(folio['premises'][1]), base_model, base_tokenizer))
print("---------------------")
print(answer(prompt_ex.format(folio['premises'][1]), align_model, align_tokenizer))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
# -----------------------------------------------------------
# Lo que sigue es una forma de automatizar la evaluación y poder medir valores como Accuracy y demás.
# No obstante la evaluación está inconclusa y no se usa para el proyecto de RL.

In [29]:
def get_answer(dataset, ans_num):
    """
    dataset = huggingface dataset 
    ans_num = int ; Valor de la entrada del dataset que queremos analizar

    Nos regresa las respuestas del modelo filtradas. 
    """
    chosen = dataset['chosen'][ans_num][1]['content']
    rejected = dataset['rejected'][ans_num][1]['content']
    return chosen, rejected

cho1, rej1 = get_answer(kuro_ds, 0)
print(cho1, '\n', '---', '\n', rej1)

∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ∀x (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina))  
 --- 
  Dependent(x) ::: x is a person dependent on caffeine.
             Drinks(x) ::: x regularly drinks coffee.
             Jokes(x) ::: x jokes about being addicted to caffeine.
             Unaware(x) ::: x is unaware that caffeine is a drug.
             Student(x) ::: x is a student.
             Premises:
             ∀x (Drinks(x) → Dependent(x)) ::: All people who regularly drink coffee are dependent on caffeine.
             ∀x (Drinks(x) ⊕ Jokes(x)) ::: People who either regularly drink coffee or joke about being addicted to caffeine.
             ∀x (Jokes(x) → ¬Unaware(x)) ::: No one who jokes about being addicted to caffeine is unaware that caffeine is a drug. 
             (

In [30]:
cho1

'∀x (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ∀x (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ∀x (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina)) '

In [31]:
# '∀x|∃x'
cho1.split('∀x ')

['',
 '(DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ',
 '(DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ',
 '(¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ¬(Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ¬(IsDependentOn(rina, caffeine) ⊕ Student(rina)) ']

In [34]:
import re
re.split('∀x|∃x|¬\(', cho1) # Casi pero no nos regesa la ER.

['',
 ' (DrinkRegularly(x, coffee) → IsDependentOn(x, caffeine)) ',
 ' (DrinkRegularly(x, coffee)  ∨ (¬WantToBeAddictedTo(x, caffeine))) ',
 ' (¬WantToBeAddictedTo(x, caffeine) → ¬AwareThatDrug(x, caffeine)) ',
 'Student(rina) ⊕  ¬AwareThatDrug(rina, caffeine)) ',
 'IsDependentOn(rina, caffeine) ⊕ Student(rina)) ']

**VER QUÉ VERGAS VAMOS A HACER PARA LA EVALUACIÓN Y LA SEPARACIÓN DE PREMISAS**